imports
-

In [1]:
import sys, os
sys.version

'3.9.13 (main, Aug 25 2022, 23:26:10) \n[GCC 11.2.0]'

In [2]:
#math
import numpy as np
#capture
import cv2
import wave
#utils
from tqdm import tqdm
from IPython.display import Image
import datetime
from collections import deque

In [3]:
%load_ext tensorboard

In [4]:
#neural
import tensorflow as tf
import keras
from tensorflow.python.client import device_lib
from tensorflow.keras import layers
print(tf.__version__)

2.9.2


In [5]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
session = tf.compat.v1.Session(config=config)

2022-10-17 02:20:10.158661: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-17 02:20:10.786323: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 399 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


system info
-

In [6]:
for i in device_lib.list_local_devices():
    print(f'{i.device_type}|{i.name} memory limit: {i.memory_limit}')
    print(i.physical_device_desc)

CPU|/device:CPU:0 memory limit: 268435456

GPU|/device:GPU:0 memory limit: 419128934
device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


2022-10-17 02:20:10.792454: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /device:GPU:0 with 399 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


import data
-

In [7]:
def reshape(frame):
    global WIDTH, HEIGHT
    newframe = np.zeros(WIDTH*HEIGHT).reshape(HEIGHT, WIDTH)
    if frame.shape[0] == HEIGHT and frame.shape[1] == WIDTH:
        return frame
    elif frame.shape[0] <= HEIGHT and frame.shape[1] <= WIDTH:
        newframe[:frame.shape[0],:frame.shape[1]] = frame
    elif frame.shape[0] <= HEIGHT:
        newframe[:frame.shape[0],:] = frame[:, :WIDTH]
    elif frame.shape[1] <= WIDTH:
        newframe[:,:frame.shape[1]] = frame[:HEIGHT, :]
    else:
        newframe = frame[:HEIGHT, :WIDTH]
    return newframe

def deinterlase(frame):
    even_fields=frame[::2,:];
    odd_fields=frame[1::2,:];
    return even_fields, odd_fields

In [8]:
WIDTH = int(720)
HEIGHT = int(625-1)

def normilizeFrame(frame):
    global WIDTH, HEIGHT
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #change dimentions(choose 1 of 2 options)
    #frame = cv2.resize(frame, (WIDTH, HEIGHT), interpolation=cv2.INTER_NEAREST)
    frame = reshape(frame)
    
    return deinterlase(frame)
    #return frame / 255.0

def retrieve():
    global WIDTH, HEIGHT
    if cap.grab():
        ret, frame = cap.retrieve()
        if not ret:
            print('error reading frame')
            return False, normilizeFrame(np.zeros(WIDTH*HEIGHT).reshape(WIDTH, HEIGHT))
        return True, normilizeFrame(frame)
    else:
        return False, normilizeFrame(np.zeros(WIDTH*HEIGHT).reshape(WIDTH, HEIGHT))

def getNext():
    global first, bottomFrame, half
    if first == True:
        first = False
        half = False
        ret, (topFrame, bottomFrame) = retrieve()
        return ret, topFrame[np.newaxis is None,:,:]
    
    if half:
        ret, (topFrame, bottomFrame) = retrieve()
        half = False
        return ret, topFrame[np.newaxis is None,:,:]
    else:
        half = True
        return True, bottomFrame[np.newaxis is None,:,:]

def openCap(file):
    global cap, gray, half, first
    first = True
    cap = cv2.VideoCapture(file + '.avi')
    if (cap.isOpened()== False): 
      print("Error opening video stream or file")
openCap('testfiles/pal/whitenosie')

In [9]:
openCap('testfiles/pal/whitenosie')


In [10]:
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(frameCount)

#tech calc

#audio in pcm have 44100 samples in second per channel, respectively we have 88200 samples
#per frame we have 44100/25(pal) = 1764 samples(for single channel)(and we take half of frame for learning /2)

samplesPerFrame = int(44100/cap.get(cv2.CAP_PROP_FPS)/2)
print(samplesPerFrame)

7501
882


create dataset

In [11]:
dir = "./dataset/"

In [12]:
def initQueues():
    global bufR0, bufL1, bufR1, bufR2, bufL2
    bufR0 = deque()
    bufL1 = deque()
    bufR1 = deque()
    bufL2 = deque()
    bufR2 = deque()
    
    for i in range(1*16):
        bufR0.append(0)
    
    for i in range(2*16):
        bufL1.append(0)
    
    for i in range(3*16):
        bufR1.append(0)
    
    for i in range(4*16):
        bufL2.append(0)
    
    for i in range(5*16):
        bufR2.append(0)    
        
def pairwise(iterable):
    a = iter(iterable)
    return zip(a, a, a, a, a, a)
    
def Abreacker(sample):
    initQueues()
    global bufR0, bufL1, bufR1, bufR2, bufL2
    filtered = np.array([], dtype=np.int16)
    line = np.zeros(6, dtype=np.int16)
    
    for elem in pairwise(sample):
        bufR0.append(elem[1])
        bufL1.append(elem[2])
        bufR1.append(elem[3])
        bufL2.append(elem[4])
        bufR2.append(elem[5])
        
        line[0] = elem[0]
        line[1] = bufR0.popleft()
        line[2] = bufL1.popleft()
        line[3] = bufR1.popleft()
        line[4] = bufL2.popleft()
        line[5] = bufR2.popleft()
                
        filtered = np.append(filtered, line)
    
    return filtered

In [13]:
mainAudioFile = wave.open(f'{dir}audio.wav', mode='r')

def gen():
    global mainAudioFile, samplesPerFrame
    
    #audio
    mainAudioFile.rewind()
    
    if not (mainAudioFile.getparams()[0] == 2 and mainAudioFile.getparams()[2] == 44100):
        print('wrong audiofile params')
        sys.exit(1)
    
    #video
    
    openCap(f'{dir}video')
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT) -1
    
    while int(cap.get(cv2.CAP_PROP_POS_FRAMES)) <= frames:
        #audio
        frame = mainAudioFile.readframes(samplesPerFrame)
        
        if len(frame) < samplesPerFrame*2:
            frame = frame + b'\x00' * ((samplesPerFrame*4)-len(frame))
        
        #video
        vframe = getNext()[1]
        
        yield vframe, (Abreacker(tf.io.decode_raw(frame, tf.int16, little_endian=True).numpy())/32767)
    

In [14]:
dataset=tf.data.Dataset.from_generator(
    gen, 
    output_signature=(tf.TensorSpec(shape=(1, int(HEIGHT/2), WIDTH), dtype=tf.uint8, name='image'),
                      tf.TensorSpec(shape=(samplesPerFrame*2,), dtype=tf.float32, name='audio'),
                      #tf.TensorSpec(shape=(1), dtype=tf.int32, name='frameNum')
                     )
)#.cache('./dataset/')#.repeat()

2022-10-17 02:20:10.854146: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 399 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


#create model
-

In [15]:
input = keras.Input(shape=(int(HEIGHT/2), WIDTH), name = 'image')

In [16]:
x = tf.keras.layers.Rescaling(1./127.5, name='normalize')(input)
i = 1
x = tf.keras.layers.Conv1D(256, 8, activation='sigmoid', name=f'conv1d_{i}')(x)
#i +=1
#x = tf.keras.layers.Conv1D(18, 1, activation='sigmoid', name=f'conv1d_{i}')(x)

print(x.shape)

x = tf.keras.layers.Reshape((x.shape[1], x.shape[2], 1))(x)

i = 1
x = tf.keras.layers.Conv2D(40, 256, activation='sigmoid', name=f'conv2d_{i}')(x)
i +=1
#x = tf.keras.layers.Conv2D(8, 64, activation='sigmoid', name=f'conv2d_{i}')(x)

print(x.shape)
print(f'diff H:{x.shape[1]-(samplesPerFrame/3)}   W:{x.shape[2]-6}    total:{x.shape[1]*x.shape[2]*x.shape[3]-(samplesPerFrame*2)}') 

(None, 305, 256)
(None, 50, 1, 40)
diff H:-244.0   W:-5    total:236


In [17]:
x = tf.keras.layers.Flatten(name='Flatten')(x)

In [18]:
x = tf.keras.layers.Dense((samplesPerFrame*2)+200, activation='sigmoid', bias_initializer='zeros', use_bias=True, name='layer_1')(x)

In [19]:
output = tf.keras.layers.Dense((samplesPerFrame*2), activation='sigmoid', bias_initializer='zeros', use_bias=True, name='audio')(x)

In [20]:
model = keras.Model(input, output)
model.summary()

#tf.keras.utils.plot_model(
#    model,
#    to_file="model.png",
#    show_shapes=True,
#    show_dtype=False,
#    show_layer_names=True,
#    rankdir="TB",
#    expand_nested=True,
#    dpi=96,
#    layer_range=None,
#    show_layer_activations=True,
#)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 312, 720)]        0         
                                                                 
 normalize (Rescaling)       (None, 312, 720)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 305, 256)          1474816   
                                                                 
 reshape (Reshape)           (None, 305, 256, 1)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 1, 40)         2621480   
                                                                 
 Flatten (Flatten)           (None, 2000)              0         
                                                                 
 layer_1 (Dense)             (None, 1964)              392996

compile and learn
-

In [21]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['binary_accuracy'])

In [22]:
testfile = 'testfiles/pal/streetSrakerffV1'

In [24]:
#сделать обратную хуеверть для аудио нормального человека

In [ ]:
loss = 1
step = 1
while loss >= 0.1:
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    fitHistory = model.fit(dataset, epochs=5, batch_size=1, callbacks=[tensorboard_callback])
    model.save(f'model-{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}', save_format="h5")
    if step%1 == 0:
        openCap(testfile)
        
        bar = tqdm(total=cap.get(cv2.CAP_PROP_FRAME_COUNT), ncols=100)
        file = wave.open(f'output/{step}.wav', mode='w')
        file.setnchannels(2) 
        file.setsampwidth(2)
        file.setframerate(44100)
        ret, vframe = getNext()
        while ret:
            file.writeframes((model.predict(vframe, verbose=0)[0]*32767*16).astype(np.int16).tobytes())
            bar.update(1)
            ret, vframe = getNext()
            if int(cap.get(cv2.CAP_PROP_POS_FRAMES)) > 25*2*15:    #limiting 30 on seconds
                ret = False
        bar.close()
        file.close()
        step = step +1
    loss = fitHistory.history['loss'][0]

Epoch 1/5


2022-10-17 02:20:12.390680: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201


15001/15001 [==============================] - 420s 28ms/step - loss: 0.2880 - binary_accuracy: 0.1361
Epoch 2/5
15001/15001 [==============================] - 413s 28ms/step - loss: 0.2880 - binary_accuracy: 0.1361
Epoch 3/5
15001/15001 [==============================] - 416s 28ms/step - loss: 0.2880 - binary_accuracy: 0.1361
Epoch 4/5
15001/15001 [==============================] - 417s 28ms/step - loss: 0.2880 - binary_accuracy: 0.1361
Epoch 5/5
15001/15001 [==============================] - 415s 28ms/step - loss: 0.2880 - binary_accuracy: 0.1361


 43%|████████████████████████▊                                | 1500/3449.0 [01:26<01:52, 17.28it/s]


Epoch 1/5
15001/15001 [==============================] - 414s 28ms/step - loss: 0.2880 - binary_accuracy: 0.1361
Epoch 2/5
15001/15001 [==============================] - 412s 27ms/step - loss: 0.2880 - binary_accuracy: 0.1361
Epoch 3/5
15001/15001 [==============================] - 414s 28ms/step - loss: 0.2880 - binary_accuracy: 0.1361
Epoch 4/5
15001/15001 [==============================] - 413s 28ms/step - loss: 0.2880 - binary_accuracy: 0.1361
Epoch 5/5
15001/15001 [==============================] - 414s 28ms/step - loss: 0.2880 - binary_accuracy: 0.1361


...

 43%|████████████████████████▊                                | 1500/3449.0 [01:26<01:52, 17.25it/s]


Epoch 1/5
15001/15001 [==============================] - 430s 29ms/step - loss: 0.2880 - binary_accuracy: 0.1361
Epoch 2/5
15001/15001 [==============================] - 440s 29ms/step - loss: 0.2880 - binary_accuracy: 0.1361
Epoch 3/5
15001/15001 [==============================] - 410s 27ms/step - loss: 0.2880 - binary_accuracy: 0.1361
Epoch 4/5
15001/15001 [==============================] - 410s 27ms/step - loss: 0.2880 - binary_accuracy: 0.1361
Epoch 5/5
15001/15001 [==============================] - 411s 27ms/step - loss: 0.2880 - binary_accuracy: 0.1361


 43%|████████████████████████▊                                | 1500/3449.0 [01:26<01:52, 17.36it/s]

Epoch 1/5


15001/15001 [==============================] - 408s 27ms/step - loss: 0.2880 - binary_accuracy: 0.1361
Epoch 2/5
 9415/15001 [=================>............] - ETA: 2:32 - loss: 0.2880 - binary_accuracy: 0.1361

In [ ]:
%tensorboard --logdir logs/fit